# DEMO: Stackoverflow dump

## CURARE data collection model

#### Type your dataset's URL or leave default value

In [2]:
import ipywidgets as widgets

In [3]:
url_tb = widgets.Text(value='https://archive.org/download/stackexchange/stats.stackexchange.com.7z', description='Dataset url:', disabled=False); url_tb

Text(value='https://archive.org/download/stackexchange/stats.stackexchange.com.7z', description='Dataset url:'…

#### Type additional information about your dataset or leve default values

In [4]:
dbprovider_tb = widgets.Text(value='Stack Exchange, Inc.', description='provider:', disabled=False); dbprovider_tb

Text(value='Stack Exchange, Inc.', description='provider:')

In [5]:
dblicense_tb = widgets.Text(value='CC BY-SA 3.0', description='license:', disabled=False); dblicense_tb

Text(value='CC BY-SA 3.0', description='license:')

In [6]:
dbauthor_tb = widgets.Text(value='Stack Exchange Community', description='author:', disabled=False); dbauthor_tb

Text(value='Stack Exchange Community', description='author:')

In [7]:
dbdesc_tb = widgets.Text(value='This is an anonymized dump of all user-contributed content on the Stack Exchange network. Each site is formatted as a separate archive consisting of XML files zipped via 7-zip using bzip2 compression. Each site archive includes Posts, Users, Votes, Comments, PostHistory and PostLinks.', 
                            description='description:', disabled=False); dbdesc_tb

Text(value='This is an anonymized dump of all user-contributed content on the Stack Exchange network. Each sit…

## Data Collection Model class creation

In [8]:
%run /test/libraries/dataCollectionModel.py

DataCollection class created!
Release class created!
Item class created!


## Harvest Data from HTTP and Unarchived it <br/> or look in local folder

In [9]:
import urllib.request
import shutil
from os import listdir
import zipfile

def dwnldUrl (_url, folder_name, file_name):            
    
    file_name = '/tmp/' + file_name # file will be downloaded in this location
    # Download the file from 'url' and save it locally under 'file_name':
    with urllib.request.urlopen(_url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)            
    
    # Unarchive data
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall('/tmp/' + folder_name) # creates folder f1db_csv if it doesn't exist
    zip_ref.close()

    listFilePath = []
    
    for f in os.listdir('/tmp/' + folder_name):     
        listFilePath.append('/tmp/' + folder_name + f)
    
    return listFilePath

def getLocalFiles(_path):    
    for f in os.listdir(_path):     
        listFilePath.append(_path + f)
    
    return listFilePath

In [10]:
selSource = widgets.Select(
    options=['Download from URL', 'Local Files'],
    value='Local Files',
    rows=3,
    description='Source:',
    disabled=False
)
selSource

Select(description='Source:', index=1, options=('Download from URL', 'Local Files'), rows=3, value='Local File…

In [11]:
dblocal_tb = widgets.Text(value='/test/stackexchange/release_09-10-18/cut/', description='path:', disabled=False); dblocal_tb

Text(value='/test/stackexchange/release_09-10-18/cut/', description='path:')

In [12]:
listFilePath = []
if selSource.value == 'Local Files':
    listFilePath = getLocalFiles(dblocal_tb.value)
    for f in listFilePath:     
        print (f)    
    
elif selSource.value == 'Download from URL':
    listFilePath = dwnldUrl("http://ergast.com/downloads/f1db_csv.zip", "f1/", "f1db_csv.zip")
    for f in listFilePath:     
        print (f)

else:
    print ('any')

/test/stackexchange/release_09-10-18/cut/Badges.csv
/test/stackexchange/release_09-10-18/cut/Comments.csv
/test/stackexchange/release_09-10-18/cut/PostHistory.csv
/test/stackexchange/release_09-10-18/cut/PostLinks.csv
/test/stackexchange/release_09-10-18/cut/Posts.csv
/test/stackexchange/release_09-10-18/cut/Tags.csv
/test/stackexchange/release_09-10-18/cut/Users.csv
/test/stackexchange/release_09-10-18/cut/Votes.csv


### Create item objects and append them to a list

In [13]:
import os 
from pyspark import SparkContext

## ITEM objects ###
itemList = []
index = 0

# Ask to introduce schema info if known, else infere schema

for f in listFilePath:  
    df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(f) # note: custom schema should be defined in deduction phase (later)
    content = df.dtypes # list N-tuple <attribute, data type>
    size = os.path.getsize(f) # get file size in Bytes
    ## Instance
    itemList.append(Item(url_tb.value, f, content, size)) # _id, name, content, size
    itemList[index].printInfo()
    print() # Add a blank space between instances
    index+=1

_id  = https://archive.org/download/stackexchange/stats.stackexchange.com.7z
name  = /test/stackexchange/release_09-10-18/cut/Badges.csv
content  = [('_Id', 'int'), ('_UserId', 'int'), ('_Name', 'string'), ('_Date', 'timestamp'), ('_Class', 'int'), ('_TagBased', 'boolean')]
size = 1746 Bytes

_id  = https://archive.org/download/stackexchange/stats.stackexchange.com.7z
name  = /test/stackexchange/release_09-10-18/cut/Comments.csv
content  = [('_Id', 'int'), ('_PostId', 'int'), ('_Score', 'int'), ('_Text', 'string'), ('_CreationDate', 'timestamp'), ('_UserId', 'int'), ('_UserDisplayName', 'string')]
size = 4766 Bytes

_id  = https://archive.org/download/stackexchange/stats.stackexchange.com.7z
name  = /test/stackexchange/release_09-10-18/cut/PostHistory.csv
content  = [('_Id', 'string'), ('_PostHistoryTypeId', 'string'), ('_PostId', 'int'), ('_RevisionGUID', 'string'), ('_CreationDate', 'timestamp'), ('_UserId', 'int'), ('_Text', 'string'), ('_Comment', 'string'), ('_UserDisplayName', 's

### Create release objects and append them in a list 

In [14]:
#------------------ RELEASE ------------------------------------------------------------------------------------------
# We assume that publication date is the same as the one which the origin server believes the resource was last modified
#conn = urllib.request.urlopen("http://ergast.com/downloads/f1db_csv.zip", timeout=30)
conn = urllib.request.urlopen(url_tb.value, timeout=30)
last_modified = conn.headers['last-modified']
publicationDate = last_modified

import os
print(os.path.splitext("path_to_file")[0])

# Get release size <br />
releaseSize = 0
for i in itemList:
    releaseSize = i.size + releaseSize

# Create a list of release objects    
releaseList = []
                           #_id, releaseNum, publicationDate, itemList, size
releaseList.append(Release(url_tb.value, 1, publicationDate, itemList, releaseSize)) 
releaseList[0].printInfo()
#--------------------------------------------------------------------------------------------------------------------

path_to_file
_id  = https://archive.org/download/stackexchange/stats.stackexchange.com.7z
releaseNum  = 1
publicationDate  = Wed, 05 Sep 2018 23:45:54 GMT
List of items = [<__main__.Item object at 0x7f8bb3be1438>, <__main__.Item object at 0x7f8bb3be17b8>, <__main__.Item object at 0x7f8bb3be1da0>, <__main__.Item object at 0x7f8bb3be9400>, <__main__.Item object at 0x7f8bb3be98d0>, <__main__.Item object at 0x7f8bb3be9a20>, <__main__.Item object at 0x7f8bb3be97b8>, <__main__.Item object at 0x7f8bb3be19b0>]
size = 24393 Bytes


### Create data collection object
Using release size since both release and collection have the same data <br />
Adding parameter manually, consider extracting them programmatically

In [15]:
#----------- COLLECTION ---------------------------------------------------------------------------------------------
# Ask for database name, provider, licence, author and description
# _id = url maybe change for a hdfs url
                                #_id, name, provider, licence, size, author, description, releaseList
dataCollection = DataCollection(url_tb.value, "stats.stackexchange", dbprovider_tb.value, dblicense_tb.value, releaseSize, dbauthor_tb.value, dbdesc_tb.value, releaseList) 
dataCollection.printInfo()
#--------------------------------------------------------------------------------------------------------------------

_id  = https://archive.org/download/stackexchange/stats.stackexchange.com.7z
name  = stats.stackexchange
provider  = Stack Exchange, Inc.
licence  = CC BY-SA 3.0
size = 24393 Bytes
author  = Stack Exchange Community
description  = This is an anonymized dump of all user-contributed content on the Stack Exchange network. Each site is formatted as a separate archive consisting of XML files zipped via 7-zip using bzip2 compression. Each site archive includes Posts, Users, Votes, Comments, PostHistory and PostLinks.
list of releases = [<__main__.Release object at 0x7f8bb3be1390>]


### Serialize data collection objet to JSON

In [16]:
import json

dataCollectionJson = json.dumps(dataCollection, default=lambda x: x.__dict__, indent=3)
dataCollectionJson = json.loads(dataCollectionJson)
print (dataCollectionJson)
print (type(dataCollectionJson))

{'_id': 'https://archive.org/download/stackexchange/stats.stackexchange.com.7z', 'name': 'stats.stackexchange', 'provider': 'Stack Exchange, Inc.', 'licence': 'CC BY-SA 3.0', 'size': 24393, 'author': 'Stack Exchange Community', 'description': 'This is an anonymized dump of all user-contributed content on the Stack Exchange network. Each site is formatted as a separate archive consisting of XML files zipped via 7-zip using bzip2 compression. Each site archive includes Posts, Users, Votes, Comments, PostHistory and PostLinks.', 'releaseList': [{'_id': 'https://archive.org/download/stackexchange/stats.stackexchange.com.7z', 'releaseNum': 1, 'publicationDate': 'Wed, 05 Sep 2018 23:45:54 GMT', 'itemList': [{'_id': 'https://archive.org/download/stackexchange/stats.stackexchange.com.7z', 'name': '/test/stackexchange/release_09-10-18/cut/Badges.csv', 'content': [['_Id', 'int'], ['_UserId', 'int'], ['_Name', 'string'], ['_Date', 'timestamp'], ['_Class', 'int'], ['_TagBased', 'boolean']], 'size'

### Store meta-data file in a MongoDB Atlas Cluster

In [17]:
import json
from pymongo import MongoClient

# Creates a client for the primary sandbox from cluster host cluster0-nlbcx.mongodb.net
client = MongoClient('cluster0-shard-00-00-nlbcx.mongodb.net:27017',
                      username='adminUser',
                      password='xpass',
                      authSource='admin',                      
                      replicaSet='Cluster0-shard-0',
                      ssl=True,
                      retryWrites=True)
db = client.test 
#db = client.test_database
db = client['stackoverflow-dump-db']
#collection = db.test_collection
collection = db['stackoverflow-stats-metadata-cut']

pid = collection.insert_one(dataCollectionJson).inserted_id  # this id can replace the url in datacollectionmodel class

pprint.pprint(collection.find_one())

#client.close()

DuplicateKeyError: E11000 duplicate key error collection: stackoverflow-dump-db.stackoverflow-stats-metadata-cut index: _id_ dup key: { : "https://archive.org/download/stackexchange/stats.stackexchange.com.7z" }